In [2]:
from os.path import join, basename, exists
import folium
import pyproj
import geehydro
import os
import pandas as pd
import numpy as np

In [3]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWhA3-I7iQhAB85Ptdq9PXEQsbYjVXHSoarJ5A1klQBp8iWTpHB_xGU

Successfully saved authorization token.


In [4]:
home_dir = '/home/zacharykeskinen/Documents/uavsar/'
data_dir = join(home_dir,'data')
imgs_dir = join(home_dir, 'imgs')
auc_dir = join(data_dir, 'aucillary')

In [5]:
left = -116.2080964390344
bottom = 43.6073401670351
top = 44.26398174447158
right = -114.8897370640344

In [6]:
loc_dir = (join(imgs_dir, os.listdir(imgs_dir)[0]))

class Image:
    def __init__(self, fp, dtype, pol, real_img):
        self.fp = fp
        self.dtype = dtype
        self.pol = pol
        self.comp = real_img
    array = None
    top = None
    bottom = None
    left = None
    right = None
    crs = None
    transform = None
    window = None
    nodata = None

img_list = []
        
for img in os.listdir(loc_dir):
    if 'csv' in img:
        desc = pd.read_csv(join(loc_dir, img))
    if 'tif' in img:
        if len(basename(img).split('.')) == 3:
            fp = join(loc_dir, img)
            real_img = 'real'
            dtype = basename(img).split('.')[-2]
            pol = np.nan
            img_list.append(Image(fp, dtype, pol, real_img))
        else:
            fp = join(loc_dir, img)
            real_img = basename(img).split('.')[-2]
            dtype = basename(img).split('.')[-3]
            pol = basename(img).split('.')[0].split('_')[-2][4:]
            img_list.append(Image(fp, dtype, pol, real_img))

In [28]:
loc_dir

'/home/zacharykeskinen/Documents/uavsar/imgs/lowman_test'

In [15]:
import rasterio as rio
from rasterio.windows import Window , from_bounds, crop


Map = folium.Map(location=[bottom,left],
                   tiles='Stamen Terrain', zoom_start = 10)

##Define IDAHO with state features in GEE
site = ee.Geometry.Polygon(
    [[[right, bottom],
      [left, bottom],
      [left, top],
      [right, top]]]);

with rio.open(img_list[1].fp) as src:
    boundary = src.bounds
    print(boundary)
    crs = src.crs
    transform_affine = src.transform
    my_proj = pyproj.Proj(str(crs).lower())#your data crs
    left_bottom = (my_proj(boundary[0],boundary[1], inverse=True))
    right_top = (my_proj(boundary[2],boundary[3], inverse=True))
    

#     left_bottom = my_proj(left,bottom, inverse=False)
#     right_top = my_proj(right,top, inverse=False)
#     win = from_bounds(left_bottom[0], left_bottom[1], right_top[0], right_top[1], src.transform)
#     bounds = rio.windows.bounds(win,src.transform)
#     img = src.read(1, window=win)
#     nodata = src.nodata

# # Overlay raster called img using add_child() function (opacity and bounding box set)
# Map.add_child(folium.raster_layers.ImageOverlay(img, opacity=1, 
#                                  bounds =[[bottom, left], [top, right]]))
##Define IDAHO with state features in GEE

#Tree values
ic_trees = ee.ImageCollection("NASA/MEASURES/GFCC/TC/v3").filterBounds(site).filterMetadata('year','equals',2015).select('tree_canopy_cover')
tree = ic_trees.mean()

#land cover
ic_lc = ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V-C3/Global").filterBounds(site).filterMetadata('system:index','equals','2019').select('discrete_classification');
assert ic_lc.size().getInfo() == 1
lc = ic_lc.first()

##USE - ee.ImageCollection("USGS/NLCD_RELEASES/2016_REL")  ??

#landforms
lf = ee.Image("CSP/ERGo/1_0/US/landforms")

#lithography
lith = ee.Image("CSP/ERGo/1_0/US/lithology")

# To see a google satellite view as a basemap
Map.setOptions('HYBRID')
Map.addLayer(site)
Map.addLayer(tree.clip(site), {'min':0,'max':50}, 'tree coverage', 0)
Map.addLayer(lc.clip(site), {'min':0,'max':200}, 'ground coverage', 1)
Map.addLayer(lf.clip(site), {'min':20, 'max':50}, 'NED land forms', 0)
Map.addLayer(lith.clip(site), {'min':5,'max':30}, 'Lithography', 0)

right = right_top[0]
top = right_top[1]
left = left_bottom[0]
bottom = left_bottom[1]

site_uav = ee.Geometry.Polygon(
    [[[right, bottom],
      [left, bottom],
      [left, top],
      [right, top]]]);
Map.addLayer(site_uav)


Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)

# Display map 
Map

BoundingBox(left=553859.6221955194, bottom=4822498.443121072, right=674122.4174766277, top=4931021.2988628335)


In [24]:
task = ee.batch.Export.image.toDrive(image=tree,  # an ee.Image object.
                                     region=site_uav,  # an ee.Geometry object.
                                     description='trees',
                                     folder='gee-downloads',
                                     fileNamePrefix='trees',
                                     scale=10,
                                     crs='EPSG:4326',
                                     maxPixels = 1e13)
task.start()

In [25]:
task = ee.batch.Export.image.toDrive(image=lc,  # an ee.Image object.
                                     region=site_uav,  # an ee.Geometry object.
                                     description='land_class',
                                     folder='gee-downloads',
                                     fileNamePrefix='land_class',
                                     scale=10,
                                     crs='EPSG:4326',
                                     maxPixels = 1e13)
task.start()

In [26]:
task = ee.batch.Export.image.toDrive(image=lith,  # an ee.Image object.
                                     region=site_uav,  # an ee.Geometry object.
                                     description='lithography',
                                     folder='gee-downloads',
                                     fileNamePrefix='lithography',
                                     scale=10,
                                     crs='EPSG:4326',
                                     maxPixels = 1e13)
task.start()

In [27]:
task = ee.batch.Export.image.toDrive(image=lf,  # an ee.Image object.
                                     region=site_uav,  # an ee.Geometry object.
                                     description='land_forms',
                                     folder='gee-downloads',
                                     fileNamePrefix='land_forms',
                                     scale=10,
                                     crs='EPSG:4326',
                                     maxPixels = 1e13)
task.start()

In [40]:
task.status()

{'state': 'READY',
 'description': 'land_classification',
 'creation_timestamp_ms': 1636499910586,
 'update_timestamp_ms': 1636499910586,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': '5NYVIHTKMA534SEPOWAQM53W',
 'name': 'projects/earthengine-legacy/operations/5NYVIHTKMA534SEPOWAQM53W'}